<a href="https://colab.research.google.com/github/AAhmadS/NLP_HW4/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Natural Language Processing-Homework4**

####**Students' names** :
>Amirahmad Shafiee<br/>
>Ali Shafiee<br/>
>Neda Fallah<br/>
####**Student number** : 99104027<br/>**Chosen task** : classification In this notebook we aim to do common but rather essential preprocess actions on some text chosen from the persian literature world. After doing so, an NLP task would be operated on the processed text.

In [4]:
!pip install -qU transformers
!pip install -qU torchtext
!pip install -qU fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 3946, done.
remote: Counting objects: 100% (1005/1005), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 3946 (delta 904), reused 862 (delta 851), pack-reused 2941
Receiving objects: 100% (3946/3946), 8.26 MiB | 30.30 MiB/s, done.
Resolving deltas: 100% (2511/2511), done.


In [6]:
import fasttext.util

In [5]:
!sudo pip install fastText/.
!cd fastText
fasttext.util.download_model('fa', if_exists='ignore')
ft = fasttext.load_model('cc.fa.300.bin')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./fastText
  Preparing metadata (setup.py) ... done
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4384288 sha256=4b351b35c90a3a40329c2a148cc9afcb829e21fc8f0d366107a9cf7f86d43b0e
  Stored in directory: /tmp/pip-ephem-wheel-cache-8lmqsdx8/wheels/8b/05/af/3cfae069d904597d44b309c956601b611bdf8967bcbe968903
Successfully built fasttext
  Attempting uninstall: fasttext
    Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2



In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd /content/drive/MyDrive/nlp_hw4

/content/drive/MyDrive/nlp_hw4


##Essential packages

In [9]:
import pandas as pd
import numpy as np
from numpy import pad
import json
import matplotlib.pyplot as plt
plt.switch_backend('agg')
%matplotlib inline
import matplotlib.ticker as ticker
from collections import Counter
from __future__ import unicode_literals, print_function, division
import random
from tqdm import tqdm
from sklearn.utils import resample
from joblib import Parallel, delayed
from sklearn import metrics
from sklearn.model_selection import train_test_split as tts
import sklearn.svm as svm
from sklearn.model_selection import StratifiedKFold as skf
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score as acs
import plotly.express as px
from sklearn.metrics import roc_curve, auc,precision_recall_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.utils.fixes import loguniform
from sklearn.metrics import f1_score
from sklearn.model_selection import learning_curve
from sklearn.metrics import classification_report
import seaborn as sns
import fasttext.util

import itertools
import torch
from torch import nn
import torchtext
from torchtext.vocab import Vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchtext.data.utils import get_tokenizer
import torch.nn.functional as F
from torch import optim

import nltk, re
from nltk import word_tokenize, sent_tokenize, WhitespaceTokenizer, FreqDist, pos_tag
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist


from transformers import pipeline, AutoTokenizer


import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os

import copy
import collections
import torch

##Load data and analysis


In [ ]:
with open('food.jsonl', 'r') as json_file:
    json_list = list(json_file)

data = pd.DataFrame([])
for json_str in json_list:
    result = json.loads(json_str)
    data = data.append(result,ignore_index=True)

In [28]:
data.isna().value_counts()

review  sentiment  category  aspects
False   False      False     False      1917
dtype: int64

In [29]:
data.sample(10)

,review,sentiment,category,aspects
646,سایز تخم مرغ و رنگ زرده ها خوبه و تازه ست,1,تخم مرغ,{'کیفیت': '1'}
1711,میوه خشک مثل پاستیل بود کشمش افتضاح بود مقدار ...,-2,تخمه و مغز طعم‌دار,{'کیفیت': '-2'}
534,عطر و طعم خوبی نداره‌ خیلی کمرنگه مثل قهوه های...,-1,قهوه,{'طعم': '-1'}
1693,با بقیه چایی ها هیچ فرقی نداره چه طعم چه رنگ,0,چای,{'طعم': '0'}
1118,این تخمه از کیفیت و طعم خوب و مناسبی بر خوردار...,2,تخمه و مغز طعم‌دار,"{'کیفیت': '1', 'طعم': '2'}"
384,تازه نبود موقع پخت پوسته شد که این نشاندهنده م...,-1,حبوبات و سویا,{'کیفیت': '-1'}
528,روغن بهار کیفیت و برند خوب و معتبر ارزش خرید داره,2,روغن,"{'کیفیت': '1', 'ارزش خرید': '1'}"
116,با خریدش دقیقا میتونید بدترین مربای عمرتون رو ...,-2,مربا,"{'کیفیت': '-2', 'طعم': '-2'}"
322,اصلا بدرد نمیخوره واقعا بی کیفیترین جنسی بود ک...,-2,روغن,"{'کیفیت': '-2', 'طعم': '-2'}"
767,خوب بود برای رژیم عالیه،مزه خوبی داره و تفاوت ...,2,تن ماهی,{'طعم': '1'}


In [69]:
data["words_count"] = [len(x.split()) for x in data["review"]]

##Essential plots

In [99]:
class distributions:

  def minmax(data, col_end = 'words_count', col_beg = 'review'):
    data[col_end] = data[col_beg].apply(lambda t: len(t.split()))

    summary = data[col_end].min(), data[col_end].max(), data[col_end].mean(), np.std(data[col_end])
    print(f" max: {summary[1]} \n min :{summary[0]}\n mean : {summary[2]}\n std : {summary[3]}")

  def word_freq(data, col_end = "words_count"):

      fig = go.Figure()

      fig.add_trace(go.Histogram(
          x=data[col_end]
      ))

      fig.update_layout(
          title_text='Distribution of word counts within data',
          xaxis_title_text='Word Count',
          yaxis_title_text='Frequency',
          bargap=0.2,
          bargroupgap=0.2)

      fig.show()

  def data_gl_than(data, less_than=25.0, greater_than=10.0, col='words_count'):
      data_length = data[col].values

      data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

      data_glt_rate = (data_glt / len(data_length)) * 100

      print(f'sentences with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

  def full_df_run(data, col_begin = 'words_count', minmax = True, freq = True, gl = True):
    if minmax :
      distributions.minmax(data)
    if freq:
      distributions.word_freq(data,col_end=col_begin)
    if gl:
      distributions.data_gl_than(data,col=col_begin)

  def word_and_freq(data, col = "tokenized"):
    tokenized_sent = data[col].values
    total_len = 0
    for i in range(len(tokenized_sent)):
      total_len += len(tokenized_sent[i])

    mp_freqdist = FreqDist(itertools.chain(*tokenized_sent))
    top20words=mp_freqdist.most_common(20)
    print ('%-16s' % 'word', '%-16s' % 'Frequency','%-16s' %  '% of the total')
    for topword in top20words:
        percent=(topword[1]/total_len)*100
        print ('%-16s' % topword[0], '%-16s' % topword[1],'%-16s' %  percent)

  def author_count(data, col_auth):
    fig = go.Figure()

    groupby_rate = data.groupby(col_auth)[col_auth].count()

    fig.add_trace(go.Bar(
        x=list(sorted(groupby_rate.index)),
        y=groupby_rate.tolist(),
        text=groupby_rate.tolist(),
        textposition='auto'
    ))

    fig.update_layout(
        title_text='Distribution of sentiments within sentences',
        xaxis_title_text='sentiment',
        yaxis_title_text='Frequency',
        bargap=0.2,
        bargroupgap=0.2)

    fig.show()

In [15]:
distributions.full_df_run(data)

 max: 50 
 min :8
 mean : 18.62336984872196
 std : 8.860481005381581


sentences with word length of greater than 10.0 and less than 25.0 includes 70.06% of the whole!


##Document classification

###Tf-Idf

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
corpus = [text for text in data["review"]]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(X)

  (0, 956)	0.279793898593645
  (0, 3382)	0.2483162405528635
  (0, 3492)	0.12253480523691855
  (0, 1385)	0.25497389445037266
  (0, 1810)	0.21481990968074424
  (0, 2070)	0.15414202873264773
  (0, 1545)	0.2483162405528635
  (0, 3445)	0.24629756772152578
  (0, 1672)	0.25742923084947067
  (0, 1149)	0.29425572997823257
  (0, 303)	0.17320870915537812
  (0, 3116)	0.184340372283323
  (0, 219)	0.17320870915537812
  (0, 2709)	0.3410506484365923
  (0, 435)	0.11416892609871536
  (0, 192)	0.10123923341720321
  (0, 2436)	0.16694767490691956
  (0, 2741)	0.35721104605979553
  (0, 4261)	0.1760606652035267
  (0, 2904)	0.12202020866938003
  (1, 1783)	0.3862477792894025
  (1, 3554)	0.36633776577724153
  (1, 1380)	0.17490038478053807
  (1, 4072)	0.30253759838218797
  (1, 233)	0.23228204720032106
  :	:
  (1915, 2358)	0.2723389186965947
  (1915, 3239)	0.2762401486767938
  (1915, 1472)	0.21949290743777408
  (1915, 3860)	0.22564443781326896
  (1915, 32)	0.1926309258787993
  (1915, 1951)	0.2762401486767938
  (19

###SVM linear kernel

In [23]:
##related to scaling and splitting

y = data['sentiment']
x = X
# x=MinMaxScaler().fit_transform(x)
x_train , x_test , y_train , y_test = tts(x,y,test_size = 0.1 , random_state = 42)

In [19]:
parameters = dict(kernel=['linear'],C = [5e-3,1e-2,5e-2,1e-1,5e-1,1])

####Preprocess data<br/>implement the cross validation function

####Train and evaluation section

In [30]:
random_model = svm.SVC(max_iter=1000,probability=True,verbose=False)
model_acc = GridSearchCV(estimator=random_model,param_grid=parameters,n_jobs=-1,cv=10,scoring='accuracy',refit=True)
random_model = svm.SVC(max_iter=1000,probability=True,verbose=False)
model_f1 = GridSearchCV(estimator=random_model,param_grid=parameters,n_jobs=-1,cv=10,scoring='f1',refit=True)
random_model = svm.SVC(max_iter=1000,probability=True,verbose=False)
model_rac = GridSearchCV(estimator=random_model,param_grid=parameters,n_jobs=-1,cv=10,scoring='roc_auc',refit=True)

In [31]:
model_acc.fit(x_train,y_train)
model_f1.fit(x_train,y_train)
model_rac.fit(x_train,y_train)
modelAcc = model_acc.best_estimator_
modelF1 = model_f1.best_estimator_
modelRac = model_rac.best_estimator_

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning:

Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning:

One or more of the test scores are non-finite: [nan nan nan nan nan nan]

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning:

One or more of the test scores are non-finite: [nan nan nan nan nan nan]



In [32]:
print(f'score of model with score set to accuracy :{model_acc.best_score_}\nparameters of model with score set to accuracy:{model_acc.best_params_}\n-------')
print(f'score of model with score set to f1 :{model_f1.best_score_}\nparameters of model with score set to f1:{model_f1.best_params_}\n-----------')
print(f'score of model with score set to roc-auc :{model_rac.best_score_}\nparameters of model with score set to roc-auc:{model_rac.best_params_}\n----------')

score of model with score set to accuracy :0.5901633284043555
parameters of model with score set to accuracy:{'C': 1, 'kernel': 'linear'}
-------
score of model with score set to f1 :nan
parameters of model with score set to f1:{'C': 0.005, 'kernel': 'linear'}
-----------
score of model with score set to roc-auc :nan
parameters of model with score set to roc-auc:{'C': 0.005, 'kernel': 'linear'}
----------


In [33]:
def roc_curve(model,title,score,x,y):
  y_score = model.predict_proba(x)[:, 1]
  fpr, tpr, thresholds = metrics.roc_curve(y, y_score)

  fig = px.area(
      x=fpr, y=tpr,
      title=f'{title}\'s ROC Curve (AUC={auc(fpr, tpr):.4f})  -- score:{score}',
      labels=dict(x='False Positive Rate', y='True Positive Rate'),
      width=700, height=500
  )
  fig.add_shape(
      type='line', line=dict(dash='dash'),
      x0=0, x1=1, y0=0, y1=1
  )

  fig.update_yaxes(scaleanchor="x", scaleratio=1)
  fig.update_xaxes(constrain='domain')
  fig.show()

roc_curve(modelAcc,"Model_acc","accuracy",x_train,y_train)
roc_curve(modelF1,"Model_f1","f1",x_train,y_train)
roc_curve(modelRac,"Model_rac","roc_auc",x_train,y_)

plt.legend()
plt.show()

ValueError: ignored

In [1]:
def prec_recall(model,title,score,x,y):

  y_score = model.predict_proba(x)[:, 1]
  precision, recall, thresholds = precision_recall_curve(y, y_score)

  fig = px.area(
      x=recall, y=precision,
      title=f'{title}\'s Precision-Recall Curve (AUC={auc(precision, recall):.4f}) -- score : {score}',
      labels=dict(x='Recall', y='Precision'),
      width=700, height=500
  )
  fig.add_shape(
      type='line', line=dict(dash='dash'),
      x0=0, x1=1, y0=1, y1=0
  )
  fig.update_yaxes(scaleanchor="x", scaleratio=1)
  fig.update_xaxes(constrain='domain')

  fig.show()


prec_recall(modelAcc,"Model_acc","accuracy",x_train,y_train)
prec_recall(modelF1,"Model_f1","f1",x_train,y_train)
prec_recall(modelRac,"Model_rac","roc_auc",x_train,y_train)

plt.legend()
plt.show()

NameError: ignored

In [2]:
def learning_curve(model,x,y,fold,title):
  train_sizes, train_scores, test_scores = learning_curve(estimator=model, X=x, y=y,cv=fold,)
  train_mean = np.mean(train_scores, axis=1)
  train_std = np.std(train_scores, axis=1)
  test_mean = np.mean(test_scores, axis=1)
  test_std = np.std(test_scores, axis=1)

  plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training Accuracy')
  plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
  plt.plot(train_sizes, test_mean, color='green', marker='+', markersize=5, linestyle='--', label='Validation Accuracy')
  plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
  plt.title(f'Learning Curve for {title}')
  plt.xlabel('Training Data Size')
  plt.ylabel('Model accuracy')
  plt.grid()
  plt.legend(loc='lower right')
  plt.show()


learning_curve(modelAcc,x_train,y_train,10,"Model_acc")
learning_curve(modelF1,x_train,y_train,10,"Model_f1")
learning_curve(modelRac,x_train,y_train,10,"Model_rac")

NameError: ignored

####Test

In [ ]:
predicted_classes = modelF1.predict(x_test)
metrics.confusion_matrix(y_test,predicted_classes)

In [ ]:
print(classification_report(y_test, predicted_classes))

###Transformer-based model

####Preprocess

In [16]:
##set word_tokenizer, sent tokenizer, normalizer, lemmatizer and other tooks to suitable and favoured persian ones
!pip install -q hazm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.8/477.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 28.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.0 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.0 which is incompatible.


In [39]:
import hazm
word_tokenizer = hazm.WordTokenizer()
normalizer = hazm.Normalizer()
sent_tokenizer = hazm.SentenceTokenizer()
lemmatizer = hazm.Lemmatizer()

#####Subsection 1: tokenizing

In [38]:
tokenized_sents = [word_tokenizer.tokenize(sent) for sent in data["review"]]
data["tokenized"] = tokenized_sents
data.sample(5)

,review,sentiment,category,aspects,tokenized
1350,کیفیت خوبی داره طعم و عطر و رنگ مناسب و خوبی ه...,2,تخمه و مغز طعم‌دار,"{'کیفیت': '1', 'طعم': '1'}","[کیفیت, خوبی, داره, طعم, و, عطر, و, رنگ, مناسب..."
1012,از انقضاش چندروز مونده بود و پلمپ شده گذاشتم ت...,-2,شیر,{},"[از, انقضاش, چندروز, مونده_بود, و, پلمپ, شده, ..."
1145,تیکه های زیری کمی رنگ زرد داشتن انگار زعفرون ی...,-1,گوشت مرغ,{},"[تیکه, های, زیری, کمی, رنگ, زرد, داشتن, انگار,..."
560,خامه نمی دهد و عطر و طعم مناسبی دارد. در کل ار...,1,قهوه,"{'طعم': '1', 'ارزش خرید': '1'}","[خامه, نمی, دهد, و, عطر, و, طعم, مناسبی, دارد,..."
452,طعمش رو دوست نداشتم ، البته این موضوع سلیقه ایه .,-1,آدامس و خوشبو کننده‌ی دهان,{'طعم': '-1'},"[طعمش, رو, دوست, نداشتم, ،, البته, این, موضوع,..."


In [40]:
distributions.word_and_freq(data,"tokenized")

word             Frequency        % of the total  
و                1624             4.34491799769912
.                1096             2.9322845600235437
به               720              1.9263183241030581
از               672              1.7978971024961876
بود              609              1.62934424913717
که               578              1.546405543516066
خیلی             565              1.5116247959975386
،                501              1.340396500521711
با               485              1.2975894266527543
این              480              1.2842122160687053
هم               452              1.2092998367980308
طعم              434              1.1611418786954546
من               381              1.0193434465045348
در               337              0.9016239933649035
داره             334              0.8935976670144742
کیفیت            318              0.8507905931455173
ولی              304              0.81333440351018
خوب              294              0.786579982342082
رو  

**In the next subsection** we will check again after normalising our data, actions like:
>omitting stopwords<br/>
>punctuations removal

#####Subsection 2: normalising

In [133]:
stop_words = ["در","را","رو","اند","بود","است","هم","آن","بوده","این","پیش","بعد","همراه","به","ز","تا","اگر","پی","پس","بی","با","یا","که","از","و"]

In [134]:
import string

def clean_text(text):
    #removing roman numbers
    pattern = r"\b(?=[MDCLXVIΙ])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})([IΙ]X|[IΙ]V|V?[IΙ]{0,3})\b\.?"
    re.sub(pattern, '&', text)
    #removing numbers
    text_nonum = re.sub(r'\d+', '', text)
    #getting rid of punctuations
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation])
    #getting rid of white spaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()

    return text_no_doublespace

def normalize_sentence(tokenized_sents, minimum_length=1, stopword_removal=True, number_removal = True):

    normalized_sents = tokenized_sents
    if stopword_removal:
        normalized_sents=[[word for word in sentence if (word not in stop_words)] for sentence in normalized_sents ]

    if minimum_length>1:
        normalized_sents= [[word for word in sentence if len(word)>minimum_length] for sentence in normalized_sents ]

    return normalized_sents

In [135]:
clean_token = [word_tokenizer.tokenize(clean_text(txt)) for txt in data["review"]]
data["clean_token"] = clean_token
normalized_sents = normalize_sentence(clean_token,)
data["normalized_sents"] = normalized_sents

In [136]:
data.sample(5)

,review,sentiment,category,aspects,tokenized,clean_token,normalized_sents,words_norm_count,words_count,label,lemmatized
155,کلا محصولات خوبی هستن ولی خوب در رنج قیمتی خودشون,1,قهوه,{},"[کلا, محصولات, خوبی, هستن, ولی, خوب, در, رنج, ...","[کلا, محصولات, خوبی, هستن, ولی, خوب, در, رنج, ...","[کلا, محصولات, خوبی, هستن, ولی, خوب, رنج, قیمت...",2,10,neutral,"[محصولات, خودشون]"
69,قبلا مرباهای ایز کیفیت بهتری داشتن اثری از انج...,-1,مربا,{'کیفیت': '-1'},"[قبلا, مرباهای, ایز, کیفیت, بهتری, داشتن, اثری...","[قبلا, مرباهای, ایز, کیفیت, بهتری, داشتن, اثری...","[قبلا, مرباهای, ایز, کیفیت, بهتری, داشتن, اثری...",1,11,very negative,[مربا]
794,سایزش خوب بود و تمیز تو پیشنهاد ویژه گرفتم راض...,1,ماهی، میگو و خاویار,{},"[سایزش, خوب, بود, و, تمیز, تو, پیشنهاد, ویژه, ...","[سایزش, خوب, بود, و, تمیز, تو, پیشنهاد, ویژه, ...","[سایزش, خوب, تمیز, تو, پیشنهاد, ویژه, گرفتم, ر...",1,11,neutral,[پیشنهاد]
1432,بنظرم طعم دارچینیش از همه طعماش بهتره ماندگاری...,1,آدامس و خوشبو کننده‌ی دهان,{'طعم': '1'},"[بنظرم, طعم, دارچینیش, از, همه, طعماش, بهتره, ...","[بنظرم, طعم, دارچینیش, از, همه, طعماش, بهتره, ...","[بنظرم, طعم, دارچینیش, همه, طعماش, بهتره, ماند...",2,10,neutral,"[دارچین, ماندگار]"
1668,لطفا این محصول رو جزو محصولات ارسال سریع سوپرم...,1,تن ماهی,{},"[لطفا, این, محصول, رو, جزو, محصولات, ارسال, سر...","[لطفا, این, محصول, رو, جزو, محصولات, ارسال, سر...","[لطفا, محصول, جزو, محصولات, ارسال, سریع, سوپرم...",3,10,neutral,"[محصولات, سوپرمارکتی, بذارید]"


In [137]:
distributions.word_and_freq(data,"normalized_sents")

word             Frequency        % of the total  
خیلی             561              1.9484579049736037
،                501              1.7400666851903308
طعم              429              1.489997221450403
من               376              1.3059183106418448
داره             331              1.1496248958043902
کیفیت            316              1.0975270908585717
ولی              300              1.0419560989163656
خوب              294              1.0211169769380384
بسته             232              0.8057793831619895
اصلا             224              0.7779938871908864
مزه              219              0.760627952208947
برای             206              0.7154765212559044
نیست             203              0.7050569602667408
هست              199              0.6911642122811893
خوبی             197              0.6842178382884134
خرید             175              0.60780772436788
قیمت             173              0.6008613503751042
های              172              0.5973881633787163

In [138]:
data["words_norm_count"] = [len(sen) for sen in data["normalized_sents"]]
distributions.data_gl_than(data,col = "words_norm_count")

sentences with word length of greater than 10.0 and less than 25.0 includes 55.76% of the whole!


#####Subsection 3: Frequency analysis

In [139]:
distributions.full_df_run(data, minmax=False,col_begin="words_norm_count")
distributions.author_count(data, "sentiment")

sentences with word length of greater than 10.0 and less than 25.0 includes 55.76% of the whole!


In [141]:
name_list = ["mixed/boerderline","very negative","negative","neutral","positive","very positive"]
labels= [name_list[int(i)+2] for i in data['sentiment']]
data["label"] = labels
for name in name_list:
  df = data[data["label"]==name]
  print(f"\nword frequency within {name}'s sentiment is as follows:\n")
  distributions.word_and_freq(df,"normalized_sents")


word frequency within mixed/boerderline's sentiment is as follows:

word             Frequency        % of the total  
خیلی             186              2.263875365141188
اصلا             183              2.227361246348588
،                168              2.044790652385589
طعم              118              1.436222005842259
من               99               1.2049659201557936
نیست             95               1.1562804284323271
مزه              92               1.1197663096397275
کیفیت            92               1.1197663096397275
بد               74               0.9006815968841284
داره             70               0.8519961051606622
ولی              61               0.7424537487828627
شده              57               0.6937682570593964
بسته             56               0.6815968841285297
نداره            53               0.6450827653359299
بوی              48               0.5842259006815969
بسیار            46               0.5598831548198637
نبود             45               0.

#####Subsection 5: lemmatization

In [142]:
lem_sents = []
for nsent in data["normalized_sents"]:
  tokens = []
  for token in nsent:
    tokens.append(lemmatizer.lemmatize(token))
  lem_sents.append(tokens)
data["lemmatized"] = lem_sents

In [143]:
data.sample(5)

,review,sentiment,category,aspects,tokenized,clean_token,normalized_sents,words_norm_count,words_count,label,lemmatized
472,اولش طعمش خوبه ولی بعد از پنج دقیقه واااقعا بد...,-2,آدامس و خوشبو کننده‌ی دهان,{'طعم': '-2'},"[اولش, طعمش, خوبه, ولی, بعد, از, پنج, دقیقه, و...","[اولش, طعمش, خوبه, ولی, بعد, از, پنج, دقیقه, و...","[اولش, طعمش, خوبه, ولی, پنج, دقیقه, واااقعا, ب...",13,18,mixed/boerderline,"[اول, طعمش, خوبه, ولی, پنج, دقیقه, واااقعا, بد..."
1601,عدس کرلیچین قاطی داره حتی تو زود پز هم نپخت و ...,-2,حبوبات و سویا,{},"[عدس, کرلیچین, قاطی, داره, حتی, تو, زود, پز, ه...","[عدس, کرلیچین, قاطی, داره, حتی, تو, زود, پز, ه...","[عدس, کرلیچین, قاطی, داره, حتی, تو, زود, پز, ن...",16,19,mixed/boerderline,"[عدس, کرلیچین, قاطی, داره, حتی, تو, زود, پز, پ..."
1547,روغن سرخ کردنی بهار کیفیت خوبی دارد سرخ کردن م...,2,روغن,"{'کیفیت': '1', 'طعم': '1'}","[روغن, سرخ, کردنی, بهار, کیفیت, خوبی, دارد, سر...","[روغن, سرخ, کردنی, بهار, کیفیت, خوبی, دارد, سر...","[روغن, سرخ, کردنی, بهار, کیفیت, خوبی, دارد, سر...",22,28,positive,"[روغن, سرخ, کردن, بهار, کیفیت, خوبی, داشت#دار,..."
1161,خیلی کوچولو بود ، من به حجمش توجه نکرده بودم ....,-1,شیر,{'ارزش خرید': '-1'},"[خیلی, کوچولو, بود, ،, من, به, حجمش, توجه, نکر...","[خیلی, کوچولو, بود, ،, من, به, حجمش, توجه, نکر...","[خیلی, کوچولو, ،, من, حجمش, توجه, نکرده_بودم, ...",10,14,very negative,"[خیلی, کوچولو, ،, من, حج, توجه, کرد#کن, ارزش, ..."
1210,تخم مرغ درشت و بدون شکستگی توی دوتا بسته به نس...,1,تخم مرغ,{'ارزش خرید': '1'},"[تخم, مرغ, درشت, و, بدون, شکستگی, توی, دوتا, ب...","[تخم, مرغ, درشت, و, بدون, شکستگی, توی, دوتا, ب...","[تخم, مرغ, درشت, بدون, شکستگی, توی, دوتا, بسته...",11,13,neutral,"[تخم, مرغ, درشت, بدون, شکستگی, تو, دوتا, بسته,..."


In [144]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F
import os

In [145]:
train, test = tts(data, test_size=0.1, random_state=1, stratify=data['sentiment'])
train, valid = tts(train, test_size=0.1, random_state=1, stratify=train['sentiment'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['lemmatized'].values.tolist(), train['sentiment'].values.tolist()
x_valid, y_valid = valid['lemmatized'].values.tolist(), valid['sentiment'].values.tolist()
x_test, y_test = test['lemmatized'].values.tolist(), test['sentiment'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

(1552, 11)
(173, 11)
(192, 11)


#### Configuration

In [146]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [148]:
MAX_LEN = 30
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 3
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/sentiment_classification_bert/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [149]:
# create a key finder based on label 2 id and id to label
name_list = ["mixed/boerderline","very negative","negative","neutral","positive","very positive"]

label2id = {label: i-2 for i, label in enumerate(name_list)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'mixed/boerderline': -2, 'very negative': -1, 'negative': 0, 'neutral': 1, 'positive': 2, 'very positive': 3}
id2label: {-2: 'mixed/boerderline', -1: 'very negative', 0: 'negative', 1: 'neutral', 2: 'positive', 3: 'very positive'}


In [150]:
# setup the tokenizer and configuration

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "-2": "mixed/boerderline",
    "-1": "very negative",
    "0": "negative",
    "1": "neutral",
    "2": "positive",
    "3": "very positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "mixed/boerderline": -2,
    "negative": 0,
    "neutral": 1,
    "positive": 2,
    "very negative": -1,
    "very positive": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



#### Input Embeddings

In [151]:
idx = np.random.randint(0, len(train))
sample_rev = train.iloc[idx]['tokenized']
sample_sent = train.iloc[idx]['sentiment']

print(f'Sample: \n{" ".join(sample_rev)}\n{sample_sent}')

Sample: 
طعم و بو بسیار عالی بود بخصوص اینکه ارگانیک هم هست ، ادم با خیال راحت مصرف میکنه
2


In [152]:
tokens = train.iloc[idx]["lemmatized"]
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  sentence: {" ".join(sample_rev)}')
print(f'  Coded_tokens: {tokens}')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

  sentence: طعم و بو بسیار عالی بود بخصوص اینکه ارگانیک هم هست ، ادم با خیال راحت مصرف میکنه
  Coded_tokens: ['طعم', 'بو', 'بسیار', 'عالی', 'بخصوص', 'اینکه', 'ارگانیک', 'هست', '،', 'ادم', 'خیال', 'راحت', 'مصرف', 'میکنه']
   Tokens: طعم بو بسیار عالی بخصوص اینکه ارگانیک هست ، ادم خیال راحت مصرف میکنه
Token IDs: [7773, 5490, 3177, 4392, 9064, 3131, 14510, 2952, 1348, 5983, 8887, 6309, 3562, 12702]


In [153]:
encoding = tokenizer.encode_plus(
    sample_sent,
    max_length=MAX_LEN,
    truncation=True,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt',  # Return PyTorch tensors
)

print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids:
tensor([[2, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]])
token_type_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]])
attention_mask:
tensor([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]])


#### Dataset

In [154]:
class SentDataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, sent, targets=None, label_list=None, max_len=MAX_LEN):
        self.sent = sent
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len


        self.label_map = {label: i-2 for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}

    def __len__(self):
        return len(self.sent)

    def __getitem__(self, item):
        sentence = str(self.sent[item])

        if self.has_target:
            target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')

        inputs = {
            'sentence': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)

        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size, label_list):
    dataset = SentDataset(
        sent=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len,
        label_list=label_list)

    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [162]:
label_list = name_list
train_data_loader = create_data_loader(train['lemmatized'].to_numpy(), train['sentiment'].to_numpy(dtype=int), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
valid_data_loader = create_data_loader(valid['lemmatized'].to_numpy(), valid['sentiment'].to_numpy(dtype=int), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test['lemmatized'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

In [163]:
sample_data = next(iter(train_data_loader))

print(sample_data.keys())

print(sample_data['sentence'])
print(sample_data['input_ids'].shape)
print(sample_data['input_ids'][0, :])
print(sample_data['attention_mask'].shape)
print(sample_data['attention_mask'][0, :])
print(sample_data['token_type_ids'].shape)
print(sample_data['token_type_ids'][0, :])
print(sample_data['targets'].shape)
print(sample_data['targets'])

dict_keys(['sentence', 'input_ids', 'attention_mask', 'token_type_ids', 'targets'])
["['کلا', 'محصولات', 'کامور', 'خوشمزه', 'س', 'مناسبه', 'افراد', 'دیابتیه']", "['نظر', 'من', 'اگه', 'کس', 'دنبال', 'یه', 'دون', 'نرمال', 'میگرده', 'واسه', 'مصرف', 'معمولی', 'روزانه', 'دون', 'بسیار', 'خوبیه']", "['کیفیت', 'بسته', 'بست#بند', 'تازگی', 'طعم', 'خوبی', 'برخورداره', 'ارزش', 'غذا', 'خرید', 'داره']", "['عطر', 'زیاده', '،', 'مزش', 'زیاد', 'تلخه', 'اصلا', 'خو', 'نیومد', 'مجبور', 'شد#شو', 'چای', 'ایرانی', 'مخلوط', 'کرد#کن', 'نا', 'بشه', 'خوردش']", "['من', 'خیلی', 'ماه', 'خور', 'عاشق', 'تن', 'ماه', '#هست', 'ولی', 'اینو', 'برای', 'بار', 'اول', 'امتحان', 'میکردم', 'پسندید#پسند', 'مجدد', 'سفارش', 'نمیدم', 'ظاهر', 'بد', '،', 'طعمش']", "['ساق', 'پوست', 'ساده', 'مرغ', 'مهیا', 'پروتئین', 'مقدار', 'کیلوگرم', 'کیفیت', 'مناسب', 'قیمت', 'صرفه', 'همچنین', 'ارسال', 'سریع', 'بسیار', 'عالی']", "['یک', 'طعم', 'عجیب', 'قریب', 'اصلا', 'شبیه', 'هنداونه', 'بود#باش', 'کاملا', 'مصتوعی']", "['طعم', 'خوبی', 'داشت#دار', 'وخو

In [164]:
sample_test = next(iter(test_data_loader))
print(sample_test.keys())

dict_keys(['sentence', 'input_ids', 'attention_mask', 'token_type_ids'])


In [165]:
class ClassifierModel(nn.Module):

    def __init__(self, config,output_size):
        super(ClassifierModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size,output_size)

    def forward(self, input_ids, attention_mask, token_type_ids):
        pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)['pooler_output']

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [166]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

In [168]:
pt_model = ClassifierModel(config=config,output_size=6)
pt_model = pt_model.to(device)

print('pt_model', type(pt_model))

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pt_model <class '__main__.ClassifierModel'>


In [169]:
for name,param in pt_model.named_parameters():
    print(name)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

#### Training

In [170]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os

import copy
import collections
import torch

In [171]:
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)

    return y, loss


def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):

            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)

            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            # calculate the batch loss
            loss = loss_fn(outputs, targets)

            # accumulate all the losses
            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)

    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss


def train_op(model,
             data_loader,
             loss_fn,
             optimizer,
             scheduler,
             step=0,
             print_every_step=100,
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None,
             clip=0.0):

    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']

        # move tensors to GPU if CUDA is available
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        # clear the gradients of all optimized variables
        optimizer.zero_grad()

        # compute predicted outputs by passing inputs to the model
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        # convert output probabilities to predicted class
        _, preds = torch.max(outputs, dim=1)

        # calculate the batch loss
        loss = loss_fn(outputs, targets)

        # accumulate all the losses
        losses.append(loss.item())

        # compute gradient of the loss with respect to model parameters
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        # perform optimization step
        optimizer.step()

        # perform scheduler step
        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)

        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))

            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss

        return eval_loss_min


    return eval_cb

In [173]:
from torch import optim
from transformers import AdamW

optimizer = optim.AdamW(pt_model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model,
        data_loader=train_data_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=scheduler,
        step=step,
        print_every_step=EEVERY_EPOCH,
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader,
        clip=CLIP)

    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

    eval_y, eval_loss = eval_op(
        model=pt_model,
        data_loader=valid_data_loader,
        loss_fn=loss_fn)

    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

Epochs... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/97 [00:00<?, ?it/s]

RuntimeError: ignored

#### Prediction

In [ ]:
def predict(model, comments, tokenizer, max_len=MAX_LEN, batch_size=batch_size):
    data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size, None)

    predictions = []
    prediction_probs = []


    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)

            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [ ]:
test_sents = test['lemmatized'].to_numpy()
preds, probs = predict(pt_model, test_sents, tokenizer, max_len=128)

print(preds.shape, probs.shape)

In [ ]:
preds

In [ ]:
preds_label = [id2label[t] for t in preds]
preds_label[0:5]

####F1 Score and report

In [ ]:
y_test, y_pred = test['sentiment'].values, preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print("--------------classification_report---------------")
print(classification_report(y_test, y_pred, target_names=label_list))

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

##Token classification

###GRU and LSTM

In [ ]:
class LSTMClassifier(nn.Module):

    def __init__(self,
        num_classes,
        batch_size=10,
        embedding_dim=100,
        hidden_dim=50,
        vocab_size=128):

        super(LSTMClassifier, self).__init__()

        initrange = 0.1

        self.num_labels = num_classes
        n = len(self.num_labels)
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.word_embeddings.weight.data.uniform_(-initrange, initrange)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=True)  # !

        print("# !")

        bi_grus = torch.nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=True)
        reverse_gru = torch.nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=False)

        self.lstm.weight_ih_l0_reverse = bi_grus.weight_ih_l0_reverse
        self.lstm.weight_hh_l0_reverse = bi_grus.weight_hh_l0_reverse
        self.lstm.bias_ih_l0_reverse = bi_grus.bias_ih_l0_reverse
        self.lstm.bias_hh_l0_reverse = bi_grus.bias_hh_l0_reverse

        bi_output, bi_hidden = bi_grus()
        reverse_output, reverse_hidden = reverse_gru()

        print("# !")

        # self.classifier = nn.Linear(hidden_dim, self.num_labels[0])
        self.classifier = nn.Linear(2 * hidden_dim, self.num_labels[0])  # !


    def repackage_hidden(h):
        """Wraps hidden states in new Tensors, to detach them from their history."""

        if isinstance(h, torch.Tensor):
            return h.detach()
        else:
            return tuple(repackage_hidden(v) for v in h)


    def forward(self, sentence, labels=None):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds)  # lstm_out - 2 tensors, _ - hidden layer
        print(lstm_out[:,-1,:])
        tag_space = self.classifier(lstm_out[:,-1,:] + lstm_out[:,-1,:])  # !  # lstm_out[:,-1,:] - 1 tensor
        logits = F.log_softmax(tag_space, dim=1)
        loss = None
        if labels:
            loss = F.cross_entropy(logits.view(-1, self.num_labels[0]), labels[0].view(-1))
        return loss, logits

###Transformer-based model